In [1]:
import cv2
import mediapipe as mp
import time
import numpy as np
import HandTrackingModule as htm
import os

##############################
wCam, hCam = 640, 480
##############################

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, wCam * 2)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, hCam * 2)
pTime = 0

folderPath = "Fingers"
myList = os.listdir(folderPath)
overlayList = []
for imPath in myList:
    image = cv2.imread(f"{folderPath}/{imPath}")
    overlayList.append(image)

detector = htm.handDetector(detectionCon= 0.75)
tipIds = [4, 8, 12, 16, 20]

while 1:
    ret, img = cap.read()
    if ret == False:
        break
    
    img = detector.findHands(img)
    lmList = detector.findPosition(img, draw= False)

    if len(lmList) != 0:
        fingers = []
        if lmList[tipIds[0]][1] > lmList[tipIds[0] - 1][1]:
            fingers.append(1)
        else:
            fingers.append(0)

        for id in range(1, 5):
            if lmList[tipIds[id]][2] < lmList[tipIds[id] - 2][2]:
                fingers.append(1)
            else:
                fingers.append(0)
        
        # print(fingers)
        totalFingerrs = fingers.count(1)
        overlayList[totalFingerrs - 1] = cv2.resize(overlayList[totalFingerrs - 1], (200, 237))
        img[0:237, 0:200] = overlayList[totalFingerrs - 1]

    cTime = time.time()
    fps = 1/(cTime - pTime)
    pTime = cTime
    cv2.putText(img, f"FPS: {int(fps)}",(400, 70),      
                cv2.FONT_HERSHEY_PLAIN, 3, (250, 0, 0), 3)
    
    cv2.rectangle(img, (25, 245), (175, 445), (0, 255, 0),
                  cv2.FILLED)
    cv2.putText(img, str(totalFingerrs), (49, 395), cv2.FONT_HERSHEY_PLAIN,
                10, (255, 0, 0), 25)

    cv2.imshow("Finger Counting", img)
    if cv2.waitKey(1) == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()